<a href="https://colab.research.google.com/github/barrettbeatrice/RFM-Analysis-Predictive-Model/blob/main/RFM_Analysis_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ecommerce-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1985%2F3404%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T170900Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D435b707a207b8cbe9a1891da209ceda60f0f34babe80c579e2af9b3851222b243c2d11b726cc3b2554218b1b44bced8f223a5937933ea723cd6416be23177a374f243484e8c19aa85ae3fa886ea9b3462357679489e91b789a1b5142edff3a0d9e1875acbedc47fac9bf7e888dc45c33d6f87b8905e244e426dc9ba520c481d703b5b731b024c32bd781960a26baca93621d7eb3b7e98ea9c4c6b18899b8b12c92c1d5dba18425a7cc6938f1fbf8e637538f8b6105d4b806731b19a136e3ba906f0b4e76820df5999ac133ff947d49f2a6c7fe3292d5e17ce9e1a15124cca37ae0bbdeb65ea257ede41cdecfbfdfc13724f7779e4f8a7626eb5763e42e54fc68'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 7548686 bytes downloaded
Downloaded and uncompressed: ecommerce-data
Data source import complete.


# Introduction
RFM (Recency, Frequency, Monetary) analysis is a well-established technique designed for behaviour-driven customer segmentation. Specifically, RFM analysis evaluates customers' recency (how long ago they made a purchase), frequency (how often they make purchases), and monetary value (how much money they spend).

RFM helps divide customers into various segments to identify those more likely to respond to promotions.

In this notebook, I will perform RFM segmentation and compare the results with those obtained using the K-means clustering algorithm on the same data.

Note: I will refer to the customer groups obtained by the RFM framework as 'segments' and those obtained with the K-means clustering algorithm as 'clusters'.

# Table of Contents
- Data Import
- Data Exploration and Cleaning
    - Checking null values
    - Duplicate values
    - Cleaning records with zero UnitPrice
    - Removing cancelled purchases
- RFM Analysis
    - RFM Framework
    - Calculating Recency, Frequency and Monetary Values
    - Exploratory Analysis of RFM Features
        - Distributions
        - Correlations
        - Box Plots
    - Assigning RFM Scores
    - RFM Segmentation
    - Analysis of Segments
- K-Means Clustering
    - Log Transformation
    - Standardising Data
    - Elbow Method
    - Silhouette Score
    - Model with 4 Clusters
    - Evaluating Clusters
        - Recency and Frequency by Cluster
        - Recency and Monetary by Cluster
- Comparing Results: RFM Segments vs. K-Mean Clusters
    - Descriptive Statistics
    - Mapping of Clusters and RFM Segments
- Conclusion
    - Example of marketing strategies for RFM segments
- References

# Data Import
This dataset contains 8 variables that correspond to:

<b> InvoiceNo </b>: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.  
<b> StockCode </b> : Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.  
<b> Description </b>: Product (item) name. Nominal.  
<b> Quantity </b>: The quantities of each product (item) per transaction. Numeric.  
<b> InvoiceDate </b>: Invoice date and time.  
<b> UnitPrice </b>: Unit price. Numeric, Product price per unit in sterling.  
<b> CustomerID </b>: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.  
<b> Country </b>: Country name. Nominal, the name of the country where each customer resides.

In [2]:
#Importing necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.cluster import KMeans
from sklearn import decomposition
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

In [6]:
data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv',
                   encoding = 'unicode_escape',
                   parse_dates = ['InvoiceDate'],
                   dtype={'CustomerID': str,'InvoiceNo': str})
data.describe()

data.head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


# Data Exploration and Cleaning
## Checking Null Values

In [8]:
data.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


There are about 135K records without a CustomerID. Our goal is to segment the customers. So, CustomerID is crucial for segmentation. Let's drop the rows where CustomerID is null.

In [9]:
data.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
data.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


Looks like the null Descriptions are gone with null CustomerIDs.

## Duplicate Values

In [10]:
print('Number of duplicates: {}'.format(data.duplicated().sum()))


Number of duplicates: 5225


In [11]:
data.drop_duplicates(inplace = True)

The data has no duplicate values now.

## Cleaning Records with Zero UnitPrice

There are 40 records that have UnitPrice equal 0. I did not notice any pattern in the records with UnitPrice==0. There were no field indicating discount or promotion.  These records may result to Monetary value equal zero. These records will be removed from the dataset.

In [12]:
data[data['UnitPrice']==0].describe()

,Quantity,InvoiceDate,UnitPrice
count,40.000000,40,40.0
mean,347.100000,2011-07-26 08:19:34.500000,0.0
min,1.000000,2010-12-05 14:02:00,0.0
25%,1.000000,2011-04-12 05:53:15,0.0
50%,4.500000,2011-08-26 14:19:00,0.0
75%,24.000000,2011-11-05 06:16:45,0.0
max,12540.000000,2011-11-25 15:57:00,0.0
std,1978.311813,NaN,0.0


In [13]:
data=data[data['UnitPrice']!=0]

The records where UnitPrice equals 0 were removed.

## Removing Cancelled Purchases
If the InvoiceNo starts from a letter 'C' that means that the purchase was cancelled. The values of such purchases should not be included in the monetary value when RFM analysis is performed. There are 8,905 records where InvoiceNo starts from 'C'.

In [ ]:
len(data[data['InvoiceNo'].str.startswith('C')])


In [ ]:
data = data[~data['InvoiceNo'].str.startswith('C')]


Now the data does not contain cancelled purchases.

# RFM Analysis

## RFM Framework

There are multiple approaches for RFM segmentation.
I will use the RFM framework with assignment of scores from 1 to 3 for each (Recency, Frequency, Monetary). Based on the assigned score the below customer segments are defined:

| Segment name | RFM score breakouts | Description |
|------------|--------------------------------------|-------------|
| Champions  | 333, 332, 323                       | The best customers  |
| Loyal      | 321, 322, 331, 232, 233               | Valuable customers that are engaged |
| Recent     | 312, 313, 311, 222, 223               | Recent customers  |
| Needs attention | 213, 221, 123, 132, 133          | A valuable customer that has not purchased recently |
| At risk    | 231, 212, 122, 131, 211               | A customer that has not purchased recently and/or tends to spend less overall |
| Inactive   | 111, 112, 113, 121                   | A lapsed customer  |

## Calculating Recency, Frequency and Monetary Values

<b> Recency </b> is the number of days since the last purchase made by a customer. To calculate Recency, we need calculated how many days passed from the current date to the date of the last purchase. Thus, we need to define the current date. The data contains purchases from 2010 and 2011. The maximum data available in the dataset is 2011-12-09. Let's assume the analysis is performed 2 days after the last purchase. We take 2 days to avoid zeros in Recency.  

In [ ]:
print(data['InvoiceDate'].max())
current_date = dt.datetime(2011,12,11)

<b> Frequency </b> shows how often a customer purchases from the company within the report’s time frame. To calculate Frequency, we will calculate number of unique invoices for each CustomerID.

<b> Monetary </b> is the total cost spend on purchases by every customer. Before calculating the Monetary, we need to calculate total cost by multiplying number of UtitPrice and Quantity.

In [ ]:
data['TotalCost'] = data['Quantity'] * data['UnitPrice']

In [ ]:
rfm = data.groupby("CustomerID").agg({"InvoiceDate" : lambda InvoiceDate :(current_date - InvoiceDate.max()).days,  # Recency
                                     "InvoiceNo" : lambda InvoiceNo: InvoiceNo.nunique(),  # Frequency
                                     "TotalCost":  lambda TotalCost: TotalCost.sum()})    # Monetary
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

## Exploratory Analysis of RFM Features


In [ ]:
rfm.describe()

### Distributions

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm, x='Recency', bins=30, kde=True)
plt.title('Histogram for Recency')
plt.xlabel('Recency')
plt.ylabel('Num of occurrences')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm, x='Frequency', bins=30, kde=True)
plt.title('Histogram for Frequency')
plt.xlabel('Frequency')
plt.ylabel('Num of occurrences')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm, x='Monetary', bins=30, kde=True)
plt.title('Histogram for Monetary')
plt.xlabel('Monetary')
plt.ylabel('Num of occurrences')
plt.show()

### Correlation

In [ ]:
corr_matrix = rfm[['Recency', 'Frequency', 'Monetary']].corr()
plt.figure(figsize=(4,4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()

There is a moderate correlation between Frequency and Monetary. This correlation is expected, as customers tend to spend more when they make purchases more frequently.

### Box Plots

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data = rfm, y='Recency')
plt.title('Box Plot for Recency')
plt.ylabel('Recency')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data = rfm, y='Frequency')
plt.title('Box Plot for Frequency')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data = rfm, y='Monetary')
plt.title('Box Plot for Monetary')
plt.ylabel('Monetary')
plt.show()

Recency, Frequency, and Monetary features have a lot of outliers. The outliers may make segmentation process more difficult.

## Assigning RFM Scores

There are multiple approaches for RFM segmentation.
I will use the RFM framework with assignment of scores from 1 to 3 for each (Recency, Frequency, Monetary). 3 is the best/highest value, and 1 is the lowest/worst value.

A final RFM score is calculated simply by combining individual RFM score numbers.  
The RFM segments will be defined using the below table   

| Segment name | RFM score breakouts | Description |
|------------|--------------------------------------|-------------|
| Champions  | 333, 332, 323                       | The best customers  |
| Loyal      | 321, 322, 331, 232, 233               | Valuable customers that are engaged |
| Recent     | 312, 313, 311, 222, 223               | Recent customers  |
| Needs attention | 213, 221, 123, 132, 133          | A valuable customer that has not purchased recently |
| At risk    | 231, 212, 122, 131, 211               | A customer that has not purchased recently and/or tends to spend less overall |
| Inactive   | 111, 112, 113, 121                   | A lapsed customer  |

In [ ]:
rfm["Recency_Score"]  = pd.qcut(rfm['Recency'], 3, [3, 2, 1])
rfm["Frequency_Score"]  = pd.qcut(rfm['Frequency'].rank(method="first"), 3, [1, 2, 3])
rfm["Monetary_Score"]  = pd.qcut(rfm['Monetary'], 3, [1, 2, 3])
rfm["RFM_Score"] = rfm["Recency_Score"].astype(str) + rfm["Frequency_Score"].astype(str) + rfm["Monetary_Score"].astype(str)
rfm.head()

## RFM Segmentation


In [ ]:
def segment_customers(rfm_score):
    if rfm_score in ['333', '332', '323']:
        return 'Champions'
    elif rfm_score in ['321', '322', '331', '232', '233']:
        return 'Loyal'
    elif rfm_score in ['312', '313', '311', '222', '223']:
        return 'Recent'
    elif rfm_score in ['213', '221', '123', '132', '133']:
        return 'Needs attention'
    elif rfm_score in ['231', '212', '122', '131', '211']:
        return 'At risk'
    elif rfm_score in ['111', '112', '113', '121']:
        return 'Inactive'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_customers)
rfm.info()

In [ ]:
rfm.reset_index(inplace=True)
rfm.groupby('Segment').agg({'CustomerID':'count'})

## Analysis of Segments

### Statistical characteristics of the segments

In [ ]:
rfm_stats = rfm[['Recency','Frequency','Monetary','Segment']].groupby('Segment').agg({'mean','std','max','min'})
format_rules = {
    ('Recency', 'mean'): '{:.2f}',
    ('Recency', 'std'): '{:.2f}',
    ('Recency', 'max'): '{:.0f}',
    ('Recency', 'min'): '{:.0f}',
    ('Monetary', 'mean'): '£{:,.2f}',
    ('Monetary', 'std'): '£{:,.2f}',
    ('Monetary', 'max'): '£{:,.0f}',
    ('Monetary', 'min'): '£{:,.0f}',
    ('Frequency', 'mean'): '{:.2f}',
    ('Frequency', 'std'): '{:.2f}',
    ('Frequency', 'max'): '{:.0f}',
    ('Frequency', 'min'): '{:.0f}',
}
table_styles = [
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.2em;'}
]

rfm_stats_style = rfm_stats.style.format(format_rules)
rfm_stats_style = rfm_stats_style.set_table_styles(table_styles)
rfm_stats_style = rfm_stats_style.set_table_styles(
    {('Monetary', 'std'): [{'selector': 'th', 'props': 'border-left: 1px solid black'},
                            {'selector': 'td', 'props': 'border-left: 1px solid black'}
                          ],
    ('Frequency', 'std'): [{'selector': 'th', 'props': 'border-left: 1px solid black'},
                            {'selector': 'td', 'props': 'border-left: 1px solid black'}
                          ]
    },
    overwrite = False,
    axis=0)
rfm_stats_style


### Number of Customers per Segment

In [ ]:
plt.figure(figsize = (16, 8))
ax = sns.countplot(data = rfm,
                   x = 'Segment',
                   palette = 'hls')
total = len(rfm.Segment)
for patch in ax.patches:
    percentage = '{:.1f}%'.format(100 * patch.get_height()/total)
    x = patch.get_x() + patch.get_width() / 2 - 0.17
    y = patch.get_y() + patch.get_height() * 1.005
    ax.annotate(percentage, (x, y), size = 14)
plt.title('Number of Customers per Segment', size = 20)
plt.xlabel('Segment', size = 16)
plt.ylabel('Count', size = 16)
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.show()

### Recency and Frequency by Segment

In [ ]:
plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=rfm, x="Recency", y="Frequency", hue="Segment", palette='tab10', s=60
)
plt.title("Recency and Frequency by Segment", size=16)
plt.xlabel("Recency", size=12)
plt.ylabel("Frequency", size=12)
plt.xticks(size=10)
plt.yticks(size=10)
plt.legend(loc="best", fontsize=12, title="Segments", title_fontsize=14)
plt.ylim(0, 30)  #the axis needs limiting because of the high number of outliers
plt.show()

- Some 'At risk' and 'Inactive' customers are mixed together
- Some 'Recent', 'Needs attention' and 'At risk' customers are mixed together

### Recency and Monetary by Segment

In [ ]:
plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=rfm, x="Recency", y="Monetary", hue="Segment", palette='tab10', s=60
)
plt.title("Recency and Monetary by Segment", size=16)
plt.xlabel("Recency", size=12)
plt.ylabel("Monetary", size=12)
plt.xticks(size=10)
plt.yticks(size=10)
plt.legend(loc="best", fontsize=12, title="Segments", title_fontsize=14)
plt.ylim(0, 10000) # the axis needs limiting because of the high number of outliers
plt.show()

- Some 'At risk' and 'Inactive' customers are mixed together
- Some 'Recent', 'Needs attention' and 'At risk' customers are mixed together
Similar observations were on the Recency and Frequency scatter plot.

### Box Plots for Segments

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(x='Segment', y='Recency', data=rfm, palette='hls')
plt.title('Recency by Segments')
plt.xlabel('Segment')
plt.ylabel('Recency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(x='Segment', y='Frequency', data=rfm, palette='hls')
plt.title('Frequency by Segments')
plt.xlabel('Segment')
plt.ylabel('Frequency')
plt.ylim(0, 30)  #the axis needs limiting because of the high number of outliers
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(x='Segment', y='Monetary', data=rfm, palette='hls')
plt.title('Monetary by Segments')
plt.xlabel('Segment')
plt.ylabel('Monetary')
plt.ylim(0, 10000) # the axis needs limiting because of the high number of outliers
plt.show()


# K-Means Clustering
K-Means clustering is a very common unsupervised learning algorithm. It scales to very large datasets and tends to work well in practice. The K-Means clustering algorithm tries to find the best way of grouping data points into k different groups, where k is a parameter given to the algorithm.  
In this section I will use elbow method and silhouette score to determine the optimal number of clusters for Recency, Frequency and Monetary.  

## Log Transformation
Recency, Frequency and Monetary values have right-skewed distributions. K-means clustering works well with symmetric distribution of variables.

In [ ]:
cols_for_clustering = ['Recency', 'Frequency', 'Monetary']
rfm_log = rfm.copy()
rfm_log[cols_for_clustering] = rfm[cols_for_clustering].apply(np.log1p)
rfm_log.describe()

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm_log, x='Recency', bins=30, kde=True)
plt.title('Histogram for Recency (Log)')
plt.xlabel('Recency')
plt.ylabel('Num of occurrences')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm_log, x='Frequency', bins=30, kde=True)
plt.title('Histogram for Frequency (Log)')
plt.xlabel('Frequency')
plt.ylabel('Num of occurrences')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(data=rfm_log, x='Monetary', bins=30, kde=True)
plt.title('Histogram for Monetary (Log)')
plt.xlabel('Monetary')
plt.ylabel('Num of occurrences')
plt.show()

## Standardising Data
K-Means clustering relies on distances between data points to assign them to clusters. If features have different scales, the algorithm may give more weight to features with larger scales, leading to biased cluster assignments. RobustScaler scales the data based on the interquartile range (IQR) to handle outliers. Recency, Frequency and Monetary features have many outliers therefore RobustScaler may be a good choice.

In [ ]:
scaler = RobustScaler()
rfm_scaled = rfm_log.copy()
rfm_scaled[cols_for_clustering] = scaler.fit_transform(rfm_log[cols_for_clustering])
rfm_scaled[cols_for_clustering].describe()

## Elbow Method

The elbow method is a technique that helps to determine the optimal number of clusters (K) for K-means clustering algorithm.

In [ ]:
X = rfm_scaled[cols_for_clustering]
inertia_list = []
for K in range(2,10):
    inertia = KMeans(n_clusters=K, random_state=7).fit(X).inertia_
    inertia_list.append(inertia)

In [ ]:
plt.figure(figsize=[7,5])
plt.plot(range(2,10), inertia_list, color=(54 / 255, 113 / 255, 130 / 255))
plt.title("Inertia vs. Number of Clusters")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia")
plt.show()

There is no clear elbow on the chart. 5 or 6 clusters may be a good number of clusters.

## Silhouette Score  
Silhouette score is a metric that used to assess the quality of clustering. A higher silhouette score indicates that the clusters are well-separated, while a lower score suggests that the clusters may overlap or are poorly defined.

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_list = []
for K in range(2,10):
    model = KMeans(n_clusters = K, random_state=7)
    clusters = model.fit_predict(X)
    s_avg = silhouette_score(X, clusters)
    silhouette_list.append(s_avg)

plt.figure(figsize=[7,5])
plt.plot(range(2,10), silhouette_list, color=(54 / 255, 113 / 255, 130 / 255))
plt.title("Silhouette Score vs. Number of Clusters")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Score")
plt.show()

The highest Silhouette score is for 2 clusters. 4 clusters seem to be a good enough choice.

## Model with 4 Clusters

In [ ]:
model = KMeans(n_clusters=4, random_state = 7)
model.fit(rfm_scaled[cols_for_clustering])
rfm_scaled['Cluster'] = model.predict(rfm_scaled[cols_for_clustering])
rfm['Cluster'] = rfm_scaled.Cluster

## Evaluating Clusters

In [ ]:
plt.figure(figsize = (8, 6))
ax = sns.countplot(data = rfm,
                   x = 'Cluster',
                   palette = 'hls')
total = len(rfm.Cluster)
for patch in ax.patches:
    percentage = '{:.1f}%'.format(100 * patch.get_height()/total)
    x = patch.get_x() + patch.get_width() / 2 - 0.17
    y = patch.get_y() + patch.get_height() * 1.005
    ax.annotate(percentage, (x, y), size = 12)
plt.title('Number of Customers per Cluster', size = 16)
plt.xlabel('Cluster', size = 12)
plt.ylabel('Count', size = 12)
plt.show()

### Recency and Frequency by Cluster

In [ ]:
plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=rfm, x="Recency", y="Frequency", hue="Cluster", palette='tab10', s=60
)
plt.title("Recency and Frequency by Cluster", size=16)
plt.xlabel("Recency", size=12)
plt.ylabel("Frequency", size=12)
plt.xticks(size=10)
plt.yticks(size=10)
plt.legend(loc="best", fontsize=12, title="Clusters", title_fontsize=14)
plt.ylim(0, 30)  #the axis needs limiting because of the high number of outliers
plt.show()

### Recency and Monetary by Cluster

In [ ]:
plt.figure(figsize=(18, 8))
sns.scatterplot(
    data=rfm, x="Recency", y="Monetary", hue="Cluster", palette='tab10', s=60
)
plt.title("Recency and Monetary by Cluster", size=16)
plt.xlabel("Recency", size=12)
plt.ylabel("Monetary", size=12)
plt.xticks(size=10)
plt.yticks(size=10)
plt.legend(loc="best", fontsize=12, title="Clusters", title_fontsize=14)
plt.ylim(0, 10000) # the axis needs limiting because of the high number of outliers
plt.show()

# Comparing Results: RFM Segments vs. K-Mean Clusters
## Descriptive Statistics

In [ ]:
rfm_stats_cl = rfm[['Recency','Frequency','Monetary','Cluster']].groupby('Cluster').agg({'mean','std','max','min'})
format_rules = {
    ('Recency', 'mean'): '{:.2f}',
    ('Recency', 'std'): '{:.2f}',
    ('Recency', 'max'): '{:.0f}',
    ('Recency', 'min'): '{:.0f}',
    ('Monetary', 'mean'): '£{:,.2f}',
    ('Monetary', 'std'): '£{:,.2f}',
    ('Monetary', 'max'): '£{:,.0f}',
    ('Monetary', 'min'): '£{:,.0f}',
    ('Frequency', 'mean'): '{:.2f}',
    ('Frequency', 'std'): '{:.2f}',
    ('Frequency', 'max'): '{:.0f}',
    ('Frequency', 'min'): '{:.0f}',
}
table_styles = [
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.2em;'}
]

rfm_stats_cl_style = rfm_stats_cl.style.format(format_rules)
rfm_stats_cl_style = rfm_stats_cl_style.set_table_styles(table_styles)
rfm_stats_cl_style = rfm_stats_cl_style.set_table_styles(
    {('Monetary', 'std'): [{'selector': 'th', 'props': 'border-left: 1px solid black'},
                            {'selector': 'td', 'props': 'border-left: 1px solid black'}
                          ],
    ('Frequency', 'std'): [{'selector': 'th', 'props': 'border-left: 1px solid black'},
                            {'selector': 'td', 'props': 'border-left: 1px solid black'}
                          ]
    },
    overwrite = False,
    axis=0)
rfm_stats_cl_style

In [ ]:
combined_stats = pd.concat([rfm_stats_cl.assign(Source='Cluster'), rfm_stats.assign(Source='Segment')])
combined_stats.head(20)

In [ ]:
combined_stats[('Recency', 'mean')].plot(kind='bar', figsize=(8, 4))
plt.title('Mean Recency by Segment / Cluster')
plt.xlabel('Segment / Cluster')
plt.ylabel('Recency')
plt.show()

In [ ]:
combined_stats[('Frequency', 'mean')].plot(kind='bar', figsize=(8, 4))
plt.title('Mean Frequency by Segment / Cluster')
plt.xlabel('Segment / Cluster')
plt.ylabel('Frequency')
plt.show()

In [ ]:
combined_stats[('Monetary', 'mean')].plot(kind='bar', figsize=(8, 4))
plt.title('Mean Monetary by Segment / Cluster')
plt.xlabel('Segment / Cluster')
plt.ylabel('Monetary')
plt.show()

## Mapping of Clusters and RFM Segments

In [ ]:
rfm_cl_seg = rfm[['Cluster', 'Segment', 'CustomerID']].groupby(['Cluster', 'Segment']).agg({'count'})
rfm_cl_seg=rfm_cl_seg.reset_index()
rfm_cl_seg.head()

In [ ]:
color_mapping = {
    'Champions':'forestgreen',
    'Loyal' : 'lightseagreen',
    'Recent' : 'deepskyblue',
    'Needs attention': 'gold',
    'At risk': 'tomato',
    'Inactive': 'mediumpurple'
    }

rfm_cluster = rfm_cl_seg[rfm_cl_seg[('Cluster', )] == 0]
plt.figure(figsize=(5, 5))
colors = sns.color_palette('hls')
plt.pie(rfm_cluster[('CustomerID', 'count')],
        labels=rfm_cluster[('Segment', )],
        autopct='%1.1f%%', startangle=140,
        colors=rfm_cluster['Segment'].map(color_mapping),
        wedgeprops=dict(edgecolor='white'))
plt.title('Cluster 0 vs. RFM Segments')
plt.show()

Cluster 0 contains mainly Loyal and Recent customers. These customers purchased recently however their mean Monetary value was low. This cluster contains 'promising' customers.

In [ ]:
rfm_cluster = rfm_cl_seg[rfm_cl_seg[('Cluster', )] == 1]
plt.figure(figsize=(5, 5))
plt.pie(rfm_cluster[('CustomerID', 'count')],
        labels=rfm_cluster[('Segment', )],
        autopct='%1.1f%%', startangle=140,
        colors=rfm_cluster['Segment'].map(color_mapping),
        wedgeprops=dict(edgecolor='white'))
plt.title('Cluster 1 vs. RFM Segments')
plt.show()

Cluster 1 has mostly Inactive and At Risk customers as per the RFM Framework.

In [ ]:
rfm_cluster = rfm_cl_seg[rfm_cl_seg[('Cluster', )] == 2]
plt.figure(figsize=(5, 5))
plt.pie(rfm_cluster[('CustomerID', 'count')],
        labels=rfm_cluster[('Segment', )],
        autopct='%1.1f%%', startangle=140,
        colors=rfm_cluster['Segment'].map(color_mapping),
        wedgeprops=dict(edgecolor='white'))
plt.title('Cluster 2 vs. RFM Segments')
plt.show()

Cluster 2 almost matches the Segment with Champions.

In [ ]:
rfm_cluster = rfm_cl_seg[rfm_cl_seg[('Cluster', )] == 3]
plt.figure(figsize=(5, 5))
plt.pie(rfm_cluster[('CustomerID', 'count')],
        labels=rfm_cluster[('Segment', )],
        autopct='%1.1f%%', startangle=140,
        colors=rfm_cluster['Segment'].map(color_mapping),
        wedgeprops=dict(edgecolor='white'))
plt.title('Cluster 3 vs. RFM Segments')
plt.show()

Cluster 3 has a mix of different RFM segments.

# Conclusion

RFM Segments are easy to interpret and provide recommendations for marketing strategies tailored to each customer segment. However, some segments are not as distinct as those created by K-means clustering.

K-Means clustering generates very distinct segments that may require interpretation.

Interestingly, the Champions segment almost matched in both methods.


## Example of marketing strategies for RFM segments
<b> Champions</b>
- Invite them first to sales, promotions, etc.
- Send them rewards or coupons for their loyalty.
- Get reviews from them.

<b> Loyal </b>
- Send them rewards or coupons for their loyalty.
- Use cross-selling or up-selling of similar past purchases or popular products.
- Get reviews from them.
- Invite them to sales, promotions, etc. right after your Champions.
- Encourage or remind them to restock items or past purchases.

<b> Recent </b>
- Convert them to a subscription and try to convert them to Champions
- Offer them a promotion within the average time to order to try to convert them.
- Encourage or remind them to restock items or past purchases.
- Use cross-selling or up-selling of similar past purchases.
- Provide testimonials or reviews from your Champions or Loyal to encourage purchases.

<b> Needs Attention </b>
- Let them know about new product releases.
- Try a winback campaign to re-engage them.
- Offer them an exclusive promotion or coupon (especially time-limited promotions).
- Personalize product recommendations based on what they purchased in the past.
- Encourage or remind them to restock items or past purchases.

<b> At Risk </b>
- Try a winback campaign to re-engage them.
- Offer them an exclusive promotion or coupon (especially time-limited promotions).
- Personalize product recommendations based on what they purchased in the past.
- They may be more price sensitive than the Needs attention group, so target with more affordable products.
- Try not to over-message as they are a churn-risk

<b> Inactive </b>
- Try a winback campaign to re-engage them.
- Offer them an exclusive promotion or coupon (especially time-limited promotions).


# References

Kaggle notebooks:  
https://www.kaggle.com/code/fabiendaniel/customer-segmentation  
https://www.kaggle.com/code/ozlemilgun/customer-segmentation-using-rfm-analysis   
https://www.kaggle.com/code/fabiendaniel/customer-segmentation  
https://www.kaggle.com/code/sercanyesiloz/crm-analytics  

Article on Investopedia:  
https://www.investopedia.com/terms/r/rfm-recency-frequency-monetary-value.asp

The original article "Optimal Selection for Direct Mail" (1995) that presented the RFM concept:  
https://www.researchgate.net/publication/227442110_Optimal_Selection_for_Direct_Mail

Customer Segmentation Strategies Used on Different Platforms:  
https://www.putler.com/rfm-analysis/   
https://help.klaviyo.com/hc/en-us/articles/17797937793179   
https://help.moengage.com/hc/en-us/articles/360037365132-RFM-Segments   


# Thank you!
Thank you for reading this notebook. Please upvote if you liked the notebook. Your comments and suggestions are highly appreciated.